# NCBI RefSeq ingestion script

This script reads the listing for the NCBI refseq via the URL https://ftp.ncbi.nih.gov/genomes/refseq/assembly_summary_refseq.txt and attempts to map the columns to Darwin Core (DwC), using any of the DwC extenesions that might apply.

In [1]:
import pandas as pd
df = pd.read_table("https://ftp.ncbi.nih.gov/genomes/refseq/assembly_summary_refseq.txt", header=1, dtype = 'object', parse_dates=['seq_rel_date'])

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261398 entries, 0 to 261397
Data columns (total 23 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   # assembly_accession       261398 non-null  object        
 1   bioproject                 261260 non-null  object        
 2   biosample                  249679 non-null  object        
 3   wgs_master                 218468 non-null  object        
 4   refseq_category            261398 non-null  object        
 5   taxid                      261398 non-null  object        
 6   species_taxid              261398 non-null  object        
 7   organism_name              261398 non-null  object        
 8   infraspecific_name         242405 non-null  object        
 9   isolate                    17724 non-null   object        
 10  version_status             261398 non-null  object        
 11  assembly_level             261398 non-null  object  

## Field mappings

Attempt to map NCBI fields to known [DwC terms](https://dwc.tdwg.org/list/), including extensions

NCBI fields described at https://ftp.ncbi.nih.gov/genomes/README_assembly_summary.txt

| # | field name | field description |
| ---- | ---- | ---- |
| 1 | "assembly_accession" | Assembly accession: the assembly accession.version reported in this field is a unique identifier for the set of sequences in this particular version of the genome assembly. |
| 2 | "bioproject" | BioProject: accession for the BioProject which produced the sequences in the genome assembly. A BioProject is a collection of biological data related to a single initiative, originating from a single organization or from a consortium. A BioProject record provides users a single place to find links to the diverse data types generated for that project. The record can be retrieved from the NCBI BioProject resource: https://www.ncbi.nlm.nih.gov/bioproject/ |
| 3 | "biosample" | BioSample: accession for the BioSample from which the sequences in the genome assembly were obtained. A BioSample record contains a description of the biological source material used in experimental assays. The record can be retrieved from the NCBI BioSample resource: https://www.ncbi.nlm.nih.gov/biosample/ |
| 4 | "wgs_master" | WGS-master: the GenBank Nucleotide accession and version for the master record of the Whole Genome Shotgun (WGS) project for the genome assembly. The master record can be retrieved from the NCBI Nucleotide resource: https://www.ncbi.nlm.nih.gov/nuccore Genome assemblies that are complete genomes, and those that are clone-based, do not have WGS-master records in which case this field will be empty. |
| 5 | "refseq_category" | RefSeq Category: whether the assembly is a reference or representative genome in the NCBI Reference Sequence (RefSeq) project classification. Values: **reference genome** - a manually selected high quality genome assembly that NCBI and the community have identified as being important as a standard against which other data are compared; **representative genome** - a genome computationally or manually selected as a representative from among the best genomes available for a species or clade that does not have a designated reference genome; **na** - no RefSeq category assigned to this assembly Prokaryotes may have more than one reference or representative genome per species. For more information see: https://www.ncbi.nlm.nih.gov/refseq/about/prokaryotes/#referencegenome Eukaryotes have no more than one reference or representative genome per species. If there are no assemblies in RefSeq for a particular eukaryotic species, then the GenBank assembly that RefSeq would select as the best available for that species will be designated as the representative genome. Viruses may have one or more reference genomes per species. The representative genome designation is not applied to viruses. |
| 6 | "taxid" | Taxonomy ID: the NCBI taxonomy identifier for the organism from which the genome assembly was derived. The NCBI Taxonomy Database is a curated classification and nomenclature for all of the organisms in the public sequence databases. The taxonomy record can be retrieved from the NCBI Taxonomy resource: https://www.ncbi.nlm.nih.gov/taxonomy/ |
| 7 | "species_taxid" | Species taxonomy ID: the NCBI taxonomy identifier for the species from which the genome assembly was derived. The species taxid will differ from the organism taxid (column 6) only when the organism was reported at a sub- species or strain level. |
| 8 | "organism_name" | Organism name: the scientific name of the organism from which the sequences in the genome assembly were derived. This name is taken from the NCBI Taxonomy record for the taxid specified in column 6. Some older taxids were assigned at the strain level and for these the organism name will include the strain. Current practice is only to assign taxids at the species level; for these the organism name will be just the species, however, the strain name will be reported in the infraspecific_name field (column 9). |
| 9 | "infraspecific_name" | Infraspecific name: the strain, breed, cultivar or ecotype of the organism from which the sequences in the genome assembly were derived. Data are reported in the form tag=value, e.g. strain=AF16. Strain, breed, cultivar and ecotype are not expected to be used together, however, if they are then they will be reported in a list separated by ", /". Empty if no strain, breed, cultivar or ecotype is specified on the genomic sequence records. |
| 10 | "isolate" | Isolate: the individual isolate from which the sequences in the genome assembly were derived. Empty if no isolate is specified on the genomic sequence records. |
| 11 | "version_status" | Version status: the release status for the genome assembly version. Values: **latest** - the most recent of all the versions for this assembly chain; **replaced** - this version has been replaced by a newer version of the assembly in the same chain; **suppressed** - this version of the assembly has been suppressed An assembly chain is the collection of all versions for the same assembly accession. |
| 12 | "assembly_level" | Assembly level: the highest level of assembly for any object in the genome assembly. Values: **Complete genome** - all chromosomes are gapless and have no runs of 10 or more ambiguous bases (Ns), there are no unplaced or unlocalized scaffolds, and all the expected chromosomes are present (i.e. the assembly is not noted as having partial genome representation). Plasmids and organelles may or may not be included in the assembly but if present then the sequences are gapless. **Chromosome** - there is sequence for one or more chromosomes. This could be a completely sequenced chromosome without gaps or a chromosome containing scaffolds or contigs with gaps between them. There may also be unplaced or unlocalized scaffolds. Scaffold - some sequence contigs have been connected across gaps to create scaffolds, but the scaffolds are all unplaced or unlocalized. **Contig** - nothing is assembled beyond the level of sequence contigs |
| 13 | "release_type" | Release type: whether this version of the genome assembly is a major, minor or patch release. Values: **Major** - changes from the previous assembly version result in a significant change to the coordinate system. The first version of an assembly is always a major release. Most subsequent genome assembly updates are also major releases. **Minor** - changes from the previous assembly version are limited to the following changes, none of which result in a significant change to the coordinate system of the primary assembly-unit: - adding, removing or changing a non-nuclear assembly-unit - dropping unplaced or unlocalized scaffolds - adding up to 50 unplaced or unlocalized scaffolds which are shorter than the current scaffold-N50 value - replacing a component with a gap of the same length Patch - the only change is the addition or modification of a patch assembly-unit. See the NCBI Assembly model web page (https://www.ncbi.nlm.nih.gov/assembly/ model/#asmb_def) for definitions of assembly-units and genome patches. |
| 14 | "genome_rep" | Genome representation: whether the goal for the assembly was to represent the whole genome or only part of it. Values: **Full** - the data used to generate the assembly was obtained from the whole genome, as in Whole Genome Shotgun (WGS) assemblies for example. There may still be gaps in the assembly. **Partial** - the data used to generate the assembly came from only part of the genome. Most assemblies have full genome representation with a minority being partial genome representation. See the Assembly help web page (https://www.ncbi.nlm.nih.gov/assembly/help/) for reasons that the genome representation would be set to partial. |
| 15 | "seq_rel_date" | Sequence release date: the date the sequences in the genome assembly were released in the International Nucleotide Sequence Database Collaboration (INSDC) databases, i.e. DDBJ, ENA or GenBank. |
| 16 | "asm_name" | Assembly name: the submitter's name for the genome assembly, when one was provided, otherwise a default name, in the form ASM#####v#, is provided by NCBI. Assembly names are not unique. |
| 17 | "submitter" | Submitter: the submitting consortium or first position if a list of organizations. The full submitter information is available in the NCBI BioProject resource: www.ncbi.nlm.nih.gov/bioproject/ |
| 18 | "gbrs_paired_asm" | GenBank/RefSeq paired assembly: the accession.version of the GenBank assembly that is paired to the given RefSeq assembly, or vice-versa. "na" is reported if the assembly is unpaired. |
| 19 | "paired_asm_comp" | Paired assembly comparison: whether the paired GenBank & RefSeq assemblies are identical or different. Values: **identical** - GenBank and RefSeq assemblies are identical; **different** - GenBank and RefSeq assemblies are not identical; **na** - not applicable since the assembly is unpaired |
| 20 | "ftp_path" | FTP path: the path to the directory on the NCBI genomes FTP site from which data for this genome assembly can be downloaded. |
| 21 | "excluded_from_refseq" | Excluded from RefSeq: reasons the assembly was excluded from the NCBI Reference Sequence (RefSeq) project, including any assembly anomalies. See: https://www.ncbi.nlm.nih.gov/assembly/help/anomnotrefseq/ |
| 22 | "relation_to_type_material" | Relation to type material: contains a value if the sequences in the genome assembly were derived from type material. Values: **assembly from type material** - the sequences in the genome assembly were derived from type material; **assembly from synonym type material** - the sequences in the genome assembly were derived from synonym type material; **assembly from pathotype material** - the sequences in the genome assembly were derived from pathovar material;** **assembly designated as neotype** - the sequences in the genome assembly were derived from neotype material; **assembly designated as reftype** - the sequences in the genome assembly were derived from reference material where type material never was available and is not likely to ever be available ICTV species exemplar - the International Committee on Taxonomy of Viruses (ICTV) designated the genome assembly as the exemplar for the virus species ICTV additional isolate - the International Committee on Taxonomy of Viruses (ICTV) designated the genome assembly an additional isolate for the virus species |
| 23 | "asm_not_live_date" | Assembly no longer live date: the date the assembly transitioned from version_status latest to either replaced or suppressed. When the assembly is in status latest, "na" is reported. |

In [5]:
# 1:1 mapping to GGBN terms
# NCBI fields described at https://ftp.ncbi.nih.gov/genomes/README_assembly_summary.txt
new_names = {
  '# assembly_accession': 'geneticAccessionNumber',
  'bioproject': 'ncbi_bioproject',
  'biosample': 'ncbi_biosample',
  'wgs_master': 'ncbi_nuccore',
  'refseq_category': 'ncbi_refseq_category',
  'taxid': 'taxonID',
  'species_taxid': 'specific_host',
  'organism_name': 'scientificName',
  'infraspecific_name': 'cloneStrain',
  'isolate': 'source_mat_id',
  'version_status': 'ncbi_version_status',
  'assembly_level': 'ncbi_assembly_level',
  'release_type': 'ncbi_release_type',
  'genome_rep': 'ncbi_genome_rep',
  'seq_rel_date': 'eventDate',
  'asm_name': 'otherCatalogNumbers',
  'submitter': 'redcordedBy',
  'gbrs_paired_asm': 'ncbi_gbrs_paired_asm',
  'paired_asm_comp': 'ncbi_paired_asm_comp',
  'ftp_path': 'geneticAccessionURI',
  'excluded_from_refseq': 'ncbi_excluded_from_refseq',
  'relation_to_type_material': 'ncbi_relation_to_type_material',
  'asm_not_live_date': 'ncbiasm_not_live_date'
}

#do rename
df.rename(
  columns = new_names, 
  inplace = True
)

In [7]:
df.head(100)

,geneticAccessionNumber,ncbi_bioproject,ncbi_biosample,ncbi_nuccore,ncbi_refseq_category,taxonID,specific_host,scientificName,infraspecificEpithet,source_mat_id,...,ncbi_genome_rep,eventDate,otherCatalogNumbers,redcordedBy,ncbi_gbrs_paired_asm,ncbi_paired_asm_comp,geneticAccessionURI,ncbi_excluded_from_refseq,ncbi_relation_to_type_material,ncbiasm_not_live_date
0,GCF_000001215.4,PRJNA164,SAMN02803731,NaN,reference genome,7227,7227,Drosophila melanogaster,NaN,NaN,...,Full,2014-08-01,Release 6 plus ISO1 MT,The FlyBase Consortium/Berkeley Drosophila Gen...,GCA_000001215.4,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,NaN,NaN,na
1,GCF_000001405.40,PRJNA168,NaN,NaN,reference genome,9606,9606,Homo sapiens,NaN,NaN,...,Full,2022-02-03,GRCh38.p14,Genome Reference Consortium,GCA_000001405.29,different,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,NaN,NaN,na
2,GCF_000001635.27,PRJNA169,NaN,NaN,reference genome,10090,10090,Mus musculus,NaN,NaN,...,Full,2020-06-24,GRCm39,Genome Reference Consortium,GCA_000001635.9,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,NaN,NaN,na
3,GCF_000001735.4,PRJNA116,SAMN03081427,NaN,reference genome,3702,3702,Arabidopsis thaliana,ecotype=Columbia,NaN,...,Full,2018-03-15,TAIR10.1,The Arabidopsis Information Resource (TAIR),GCA_000001735.2,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,NaN,NaN,na
4,GCF_000001905.1,PRJNA70973,SAMN02953622,AAGU00000000.3,representative genome,9785,9785,Loxodonta africana,NaN,ISIS603380,...,Full,2009-07-15,Loxafr3.0,Broad Institute,GCA_000001905.1,different,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,NaN,NaN,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,GCF_000007065.1,PRJNA224116,SAMN02603290,NaN,representative genome,192952,2209,Methanosarcina mazei Go1,strain=Go1,NaN,...,Full,2002-05-20,ASM706v1,Gottingen Genomics Laboratory,GCA_000007065.1,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,NaN,NaN,na
96,GCF_000007085.1,PRJNA224116,SAMN02603009,NaN,representative genome,273068,911092,Caldanaerobacter subterraneus subsp. tengconge...,strain=MB4,NaN,...,Full,2002-05-14,ASM708v1,Beijing Center.HGP,GCA_000007085.1,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,NaN,assembly from type material,na
97,GCF_000007105.1,PRJNA224116,SAMN02603579,NaN,representative genome,264203,542,Zymomonas mobilis subsp. mobilis ZM4 = ATCC 31821,strain=ZM4,NaN,...,Full,2010-01-12,ASM710v1,Macrogen Inc.,GCA_000007105.1,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,NaN,NaN,na
98,GCF_000007125.1,PRJNA224116,SAMN02603416,NaN,representative genome,224914,29459,Brucella melitensis bv. 1 str. 16M,strain=16M,NaN,...,Full,2001-12-28,ASM712v1,Integrated Genomics,GCA_000007125.1,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,NaN,assembly from type material,na


In [5]:
data.to_csv('assembly_summary_refseq.csv', index= None)

In [6]:
df = pd.read_csv("assembly_summary_refseq.csv")

In [ ]:
df

In [8]:
dictn = {"Category 3": "1km",
       "Category 2": "10km",
       "Category 1": "WITHHOLD"
}


In [9]:
select = ['Category 3','Category 2','Category 1']

In [11]:
df['generalisation'] = df['sensitivityClass']

In [12]:
df['generalisation'] = df['generalisation'].replace(dictn)

In [16]:
sensitive = df[(df['speciesID'] == df['taxonID']) & df['sensitivityClass'].isin(select)]

In [17]:
sensitive

,dcterms_rightsHolder,dcterms_rights,dcterms_language,dcterms_type,dcterms_modified,dcterms_available,speciesID,taxonRank,kingdomID,kingdom,...,protectedInNSW,sensitivityClass,TSProfileID,countryConservation,highThreatWeed,widelyCultivatedNativeSpecies,CAMBA,JAMBA,ROKAMBA,generalisation
5,"NSW Dept of Planning, Industry and Environment",CC-BY 4.0,en,service,2011-11-11T11:23:09+11:00,1995-12-15T12:48:15+11:00,6,Species,138,Animalia,...,True,Category 3,10975.0,Endangered,NaN,NaN,False,False,False,1km
159,"NSW Dept of Planning, Industry and Environment",CC-BY 4.0,en,service,2011-11-11T11:23:06+11:00,1995-12-15T12:49:15+11:00,162,Species,138,Animalia,...,True,Category 3,10562.0,Not Listed,NaN,NaN,False,False,False,1km
326,"NSW Dept of Planning, Industry and Environment",CC-BY 4.0,en,service,2012-07-23T18:04:52.683+10:00,1995-12-15T12:50:21+11:00,333,Species,138,Animalia,...,True,Category 3,10821.0,Not Listed,NaN,NaN,False,False,False,1km
356,"NSW Dept of Planning, Industry and Environment",CC-BY 4.0,en,service,2011-11-11T11:23:06+11:00,1995-12-15T12:50:32+11:00,363,Species,138,Animalia,...,True,Category 3,10561.0,Not Listed,NaN,NaN,False,False,False,1km
382,"NSW Dept of Planning, Industry and Environment",CC-BY 4.0,en,service,2011-11-11T11:23:04+11:00,1995-12-15T12:50:42+11:00,390,Species,138,Animalia,...,True,Category 2,10413.0,Vulnerable,NaN,NaN,False,False,False,10km
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21986,"NSW Dept of Planning, Industry and Environment",CC-BY 4.0,en,service,2020-01-23T17:14:13.01+11:00,2019-06-24T10:36:48.61+10:00,23014,Subspecies,139,Plantae,...,False,Category 3,20148.0,Critically Endangered,NaN,NaN,False,False,False,1km
22658,"NSW Dept of Planning, Industry and Environment",CC-BY 4.0,en,service,2020-06-23T19:56:43.85+10:00,2020-06-23T15:48:30.817+10:00,23737,Species,139,Plantae,...,True,Category 2,NaN,Not Listed,NaN,NaN,False,False,False,10km
22659,"NSW Dept of Planning, Industry and Environment",CC-BY 4.0,en,service,2020-06-23T19:51:08.733+10:00,2020-06-23T19:47:12.607+10:00,23738,Species,139,Plantae,...,True,Category 2,20365.0,Endangered,NaN,NaN,False,False,False,10km
23131,"NSW Dept of Planning, Industry and Environment",CC-BY 4.0,en,service,2021-12-31T11:53:58.467+11:00,2021-03-17T11:44:46.88+11:00,24268,Species,139,Plantae,...,False,Category 2,10668.0,Not Listed,NaN,NaN,False,False,False,10km


In [18]:
sensitive = sensitive[['taxonRank', 'kingdom' ,'class','order','family','genus','scientificName','specificEpithet','vernacularName','establishmentMeans','stateConservation','protectedInNSW','sensitivityClass','TSProfileID','countryConservation','dcterms_modified','speciesID','taxonID','generalisation']]

In [23]:
sensitive['status'] = sensitive['stateConservation']

/Users/cha801/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [30]:
sensitive = sensitive.rename(columns={"stateConservation": "sourceStatus"})

In [33]:
sensitive

,taxonRank,kingdom,class,order,family,genus,scientificName,specificEpithet,vernacularName,establishmentMeans,sourceStatus,protectedInNSW,sensitivityClass,TSProfileID,countryConservation,dcterms_modified,speciesID,taxonID,generalisation,status
5,Species,Animalia,Aves,Psittaciformes,Cacatuidae,Callocephalon,Callocephalon fimbriatum,fimbriatum,Gang-gang Cockatoo,"Alive in NSW, Native",Vulnerable,True,Category 3,10975.0,Endangered,2011-11-11T11:23:09+11:00,6,6,1km,Vulnerable
159,Species,Animalia,Aves,Strigiformes,Strigidae,Ninox,Ninox strenua,strenua,Powerful Owl,"Alive in NSW, Native",Vulnerable,True,Category 3,10562.0,Not Listed,2011-11-11T11:23:06+11:00,162,162,1km,Vulnerable
326,Species,Animalia,Aves,Strigiformes,Tytonidae,Tyto,Tyto tenebricosa,tenebricosa,Sooty Owl,"Alive in NSW, Native",Vulnerable,True,Category 3,10821.0,Not Listed,2012-07-23T18:04:52.683+10:00,333,333,1km,Vulnerable
356,Species,Animalia,Aves,Strigiformes,Strigidae,Ninox,Ninox connivens,connivens,Barking Owl,"Alive in NSW, Native",Vulnerable,True,Category 3,10561.0,Not Listed,2011-11-11T11:23:06+11:00,363,363,1km,Vulnerable
382,Species,Animalia,Reptilia,Squamata,Elapidae,Hoplocephalus,Hoplocephalus bungaroides,bungaroides,Broad-headed Snake,"Alive in NSW, Native",Endangered,True,Category 2,10413.0,Vulnerable,2011-11-11T11:23:04+11:00,390,390,10km,Endangered
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21986,Subspecies,Plantae,Flora,Flora,Poaceae,Anthosachne,Anthosachne kingiana subsp. kingiana,kingiana,Philip Island Wheat Grass,"Alive in NSW, Native",Critically Endangered,False,Category 3,20148.0,Critically Endangered,2020-01-23T17:14:13.01+11:00,23014,23014,1km,Critically Endangered
22658,Species,Plantae,Flora,Flora,Orchidaceae,Caladenia,Caladenia tensa,tensa,Rigid Spider-orchid,"Alive in NSW, Native",Not Listed,True,Category 2,NaN,Not Listed,2020-06-23T19:56:43.85+10:00,23737,23737,10km,Not Listed
22659,Species,Plantae,Flora,Flora,Orchidaceae,Caladenia,Caladenia atroclavia,atroclavia,Black-clubbed Spider-orchid,"Alive in NSW, Native",Not Listed,True,Category 2,20365.0,Endangered,2020-06-23T19:51:08.733+10:00,23738,23738,10km,Not Listed
23131,Species,Plantae,Flora,Flora,Orchidaceae,Prasophyllum,Prasophyllum sandrae,sandrae,Majors Creek Leek Orchid,"Alive in NSW, Native",Critically Endangered,False,Category 2,10668.0,Not Listed,2021-12-31T11:53:58.467+11:00,24268,24268,10km,Critically Endangered


In [34]:
sensitive.to_csv('sensitive-NSW.csv',index = False,encoding='utf-8-sig')